In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostRanker, Pool
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from scipy.stats import spearmanr

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('../data/cleaned_data.csv')

In [4]:
df.head()

,Pla.,Dist.,track_condition,RaceClass,gate_position,Rtg.,Trainer,Jockey,Win Odds,Act.Wt.,Declar.Horse Wt.,Horse_id,Import type,Sire,Dam,Dam sire,race_index,rc,track,course,origin,age,colour,sex,B,BO,CC,CP,CO,E,H,P,PC,PS,SB,SR,TT,V,VO,XB
0,11,1000,G,5,9,37,K L Man,Y L Chung,154.0,129,1019,J006,PPG,Capitalist,Dorodansa,Bellamy Road,142301024,HV,Turf,C+3,AUS,NaN,Chestnut,Gelding,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,12,1200,G,5,2,39,K L Man,E C W Wong,135.0,126,1023,J006,PPG,Capitalist,Dorodansa,Bellamy Road,48250924,HV,Turf,C,AUS,NaN,Chestnut,Gelding,False,False,False,False,False,False,True,False,True,False,False,False,True,False,False,False
2,10,1200,GF,4,6,44,T P Yung,H T Mo,205.0,117,1013,J006,PPG,Capitalist,Dorodansa,Bellamy Road,661110524,ST,Turf,C,AUS,NaN,Chestnut,Gelding,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False
3,14,1600,G,4,12,47,T P Yung,C L Chau,221.0,121,1029,J006,PPG,Capitalist,Dorodansa,Bellamy Road,468030324,ST,Turf,B+2,AUS,NaN,Chestnut,Gelding,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
4,12,1200,G,4,2,50,T P Yung,K C Leung,67.0,128,1019,J006,PPG,Capitalist,Dorodansa,Bellamy Road,403070224,HV,Turf,B,AUS,NaN,Chestnut,Gelding,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False


# Utility function

In [69]:
def reduce_cardinality(series, min_freq = 10):
    counts = series.value_counts()
    rare = counts[counts < min_freq].index
    return series.apply(lambda x: 'unknown' if x in rare else x)

In [24]:
def encode_placing(group):
    finish_mask = group['Pla.'].astype(str).str.isdigit()
    max_rank = group.loc[finish_mask, 'Pla.'].astype(int).max()
    return group['Pla.'].apply(lambda x: int(x) if str(x).isdigit() else max_rank + 1)

# Approach 1 (training with all variables)

In [25]:
df1 = df.copy()

In [26]:
df1 = df1[~df1['Pla.'].isin(['UR', 'FE', 'TNP', 'PU', 'DNF', 'DISQ'])]

In [27]:
# define target and groupings
target = 'Pla.'
group_col = 'race_index'

# define categorical variables
categorical_cols = [
    'Dist.', 'track_condition', 'RaceClass', 'gate_position', 'Trainer', 'Jockey', 'Import type', 'Sire', 'Dam', 'Dam sire', 'rc', 'track', 'course', 'origin', 'age', 'colour', 'sex'
]

# define numerical variables
numerical_cols = [
    'Rtg.', 'Win Odds', 'Act.Wt', 'Declar.Horse Wt.'
]

In [28]:
# missingness indicator for 'age'
df1['age_missing'] = df1['age'].isnull().astype(int)

# impute age with median
age_median = df1['age'].median()
df1['age'] = df1['age'].fillna(age_median)

In [29]:
# fill missing categoricals with 'unknown'
for col in categorical_cols:
    df1[col] = df1[col].fillna('unknown')

# cardinality of the categories being reduced
for col in categorical_cols:
    df1[col] = reduce_cardinality(df1[col])

for col in categorical_cols:
    df1[col] = df1[col].astype(str)

cardinality in Trainer, Jockey, Sire, Dam, Dam sire, and course reduced 

In [30]:
numerical_cols_updated = ['age_missing']

for col in numerical_cols_updated:
    df1[col] = pd.to_numeric(df1[col], errors = 'coerce').fillna(0)

In [31]:
# define the features, target and groups
X = df1[categorical_cols + numerical_cols_updated]
y = pd.to_numeric(df1[target], errors = 'coerce') # ensure target is int
groups = df1[group_col]

In [32]:
# split by race index for train and test sets
unique_races = groups.unique()
train_races, test_races = train_test_split(unique_races, test_size = 0.2, random_state = 42)

train_mask = groups.isin(train_races)
test_mask = groups.isin(test_races)

X_train, y_train, group_train = X[train_mask], y[train_mask], groups[train_mask]
X_test, y_test, group_test = X[test_mask], y[test_mask], groups[test_mask]

In [33]:
# sort by group_id for requirements
train_sorted_idx = group_train.argsort()
X_train = X_train.iloc[train_sorted_idx]
y_train = y_train.iloc[train_sorted_idx]
group_train = group_train.iloc[train_sorted_idx]

test_sorted_idx = group_test.argsort()
X_test = X_test.iloc[test_sorted_idx]
y_test = y_test.iloc[test_sorted_idx]
group_test = group_test.iloc[test_sorted_idx]

In [34]:
train_pool = Pool(
    data = X_train,
    label = y_train,
    group_id = group_train,
    cat_features = categorical_cols
)

test_pool = Pool(
    data = X_test,
    label = y_test,
    group_id = group_test,
    cat_features = categorical_cols
)

In [35]:
# Initialize and train CatBoost ranking model
model = CatBoostRanker(
    iterations=1000,
    learning_rate=0.02,
    depth=3,
    loss_function='YetiRank',
    eval_metric='NDCG',
    early_stopping_rounds=50,
    random_seed=42,
    verbose=100
)

In [36]:
model.fit(train_pool, eval_set=test_pool)

Groupwise loss function. OneHotMaxSize set to 10
0:	test: 0.8082862	best: 0.8082862 (0)	total: 20.7ms	remaining: 20.7s
100:	test: 0.8988427	best: 0.8993684 (97)	total: 952ms	remaining: 8.47s
200:	test: 0.9041507	best: 0.9041507 (200)	total: 2.01s	remaining: 7.98s
300:	test: 0.9080305	best: 0.9080305 (300)	total: 2.99s	remaining: 6.94s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9088402074
bestIteration = 316

Shrink model to first 317 iterations.


In [ ]:
test_preds = model.predict(X_test)

# Compute predicted ranks within each group (race)
test_results = X_test.copy()
test_results['true_pla'] = y_test
test_results['pred_score'] = test_preds
test_results['race_index'] = group_test

test_results['pred_rank'] = test_results.groupby('race_index')['pred_score'].rank(ascending=False, method='min')

In [39]:
# Evaluate using Spearman rank correlation per race
def race_spearman(group):
    if len(group) <= 1:
        return np.nan
    return spearmanr(group['true_pla'], group['pred_rank']).correlation

spearman_scores = test_results.groupby('race_index').apply(race_spearman)
mean_spearman = spearman_scores.dropna().mean()
print(f'Mean Spearman rank correlation on test races: {mean_spearman:.4f}')

Mean Spearman rank correlation on test races: -0.4355


/var/folders/rn/zdwl9_vx4yz5r87jv7jnztdh0000gn/T/ipykernel_1368/1274675035.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  spearman_scores = test_results.groupby('race_index').apply(race_spearman)


# Updated approach (include unfinished horses)

In [2]:
# encode placing including unfinished horses with large numbers
def encode_placing(group):
    finish_mask = group['Pla.'].astype(str).str.isdigit()
    max_rank = group.loc[finish_mask, 'Pla.'].astype(int).max()
    return group['Pla.'].apply(lambda x: int(x) if str(x).isdigit() else max_rank + 1)

In [ ]:
# define target and groupings
target = 'Pla.'
group_col = 'race_index'

# define categorical variables
categorical_cols = [
    'Dist.', 'track_condition', 'RaceClass', 'gate_position', 'Trainer', 'Jockey', 'Import type', 'Sire', 'Dam', 'Dam sire', 'rc', 'track', 'course', 'origin', 'age', 'colour', 'sex'
]

# define numerical variables
numerical_cols = [
    'Rtg.', 'Win Odds', 'Act.Wt', 'Declar.Horse Wt.'
]